# Analysis on 2D Heisenberg model

In [1]:
import multiprocessing
import sys
import os
from pathlib import Path
from time import time
from typing import Tuple
from socket import gethostname

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
from tqdm.notebook import tqdm

# set plot customisation
plt.rc('font',**{'family':'serif',
             'serif': 'cmr10',
             'size': 12})
plt.rc('mathtext',**{'fontset':'cm'})
plt.rc('axes', unicode_minus=False, linewidth=1.2, xmargin=0,
           prop_cycle=cycler('color', ['#008fd5',
          '#fc4f30', '#e5ae38', '#6d904f', '#8b8b8b', '#810f7c']))

from heisenberg_2d import initialize, metropolis, energy, magnetization, run_simulation

## Run the routine

In [2]:
# Settings

n_temp = 500
data_path = './results'

N = 10
H = 1
steps = 4000
temp = np.linspace(0.3, 10.5, n_temp)

In [3]:
*results, wall_time = run_simulation(N, H, steps, temp)
results = np.array(results).T

Using 80 threads on brute



Took 45.48704242706299 s


### Save results

In [4]:
# mkdir in case
if not os.path.exists(data_path):
    os.makedirs(data_path)

# gather results in a table just in case
df = pd.DataFrame(results, columns=['E', 'M', 'C', 'X'])
df['temp'] = temp
for name, var in zip(['n_temp', 'N', 'steps', 'H', 'wall_time'], [n_temp, N, steps, H, wall_time]):
    df[name] = var
    
# save the table
df.to_csv(f'{data_path}/data_{n_temp}_{N}_{steps}_{H}.csv', sep=',', header=True, index=False)
display(df)

,E,M,C,X,temp,n_temp,N,steps,H,wall_time
0,-0.735318,0.990125,0.017533,0.003530,0.300000,500,10,4000,1,45.487042
1,-0.697952,0.920723,0.026942,0.055755,0.320441,500,10,4000,1,45.487042
2,-0.719808,0.939449,0.095537,0.049517,0.340882,500,10,4000,1,45.487042
3,-0.773663,0.961333,0.033885,0.017992,0.361323,500,10,4000,1,45.487042
4,-0.786199,1.096441,0.015060,0.011704,0.381764,500,10,4000,1,45.487042
...,...,...,...,...,...,...,...,...,...,...
495,-0.066298,0.116650,0.002661,0.108390,10.418236,500,10,4000,1,45.487042
496,-0.069380,0.119261,0.002710,0.109892,10.438677,500,10,4000,1,45.487042
497,-0.067792,0.118758,0.002964,0.112726,10.459118,500,10,4000,1,45.487042
498,-0.063707,0.104054,0.002637,0.110939,10.479559,500,10,4000,1,45.487042


# Show all results

In [5]:
data_path = './results'

dfs = [pd.read_csv(f) for f in Path.cwd().rglob(f'{data_path}/*.csv')]
df = pd.concat(dfs, axis=0, join='inner').sort_index()

In [6]:
df

,E,M,C,X,temp,n_temp,N,steps,H,wall_time
0,-0.735318,0.990125,0.017533,0.003530,0.300000,500,10,4000,1,45.487042
0,-0.673370,0.918997,0.010483,0.014668,0.300000,200,10,2000,1,25.487375
0,-0.692639,0.893443,0.143314,0.061407,0.300000,200,25,2000,1,42.088852
0,-0.708981,0.926891,0.765390,0.363171,0.300000,200,50,2000,1,104.371786
0,-0.687219,0.835627,0.033827,0.026353,0.300000,100,20,2000,1,27.794020
...,...,...,...,...,...,...,...,...,...,...
495,-0.066298,0.116650,0.002661,0.108390,10.418236,500,10,4000,1,45.487042
496,-0.069380,0.119261,0.002710,0.109892,10.438677,500,10,4000,1,45.487042
497,-0.067792,0.118758,0.002964,0.112726,10.459118,500,10,4000,1,45.487042
498,-0.063707,0.104054,0.002637,0.110939,10.479559,500,10,4000,1,45.487042


## Plotting

Example of plotting all simulation data with a specific size $N$

In [7]:
df[df['N'] == 10]

,E,M,C,X,temp,n_temp,N,steps,H,wall_time
0,-0.735318,0.990125,0.017533,0.003530,0.300000,500,10,4000,1,45.487042
0,-0.673370,0.918997,0.010483,0.014668,0.300000,200,10,2000,1,25.487375
1,-0.697952,0.920723,0.026942,0.055755,0.320441,500,10,4000,1,45.487042
1,-0.692783,0.932649,0.106530,0.097570,0.351256,200,10,2000,1,25.487375
2,-0.776206,0.986180,0.011772,0.025485,0.402513,200,10,2000,1,25.487375
...,...,...,...,...,...,...,...,...,...,...
495,-0.066298,0.116650,0.002661,0.108390,10.418236,500,10,4000,1,45.487042
496,-0.069380,0.119261,0.002710,0.109892,10.438677,500,10,4000,1,45.487042
497,-0.067792,0.118758,0.002964,0.112726,10.459118,500,10,4000,1,45.487042
498,-0.063707,0.104054,0.002637,0.110939,10.479559,500,10,4000,1,45.487042


In [8]:
E, M, C, X, temp, *_ = df[df['N'] == 10].values.T

In [9]:
## then we plot a figure with energy, magnetization, specific heat and susceptibility
if len(snaps)==0:
    plt.figure(figsize=(12,12))

    plt.subplot(2,2,1)
    plt.title('external field H={}'.format(H))
    plt.plot(temp, E, 'ro', markeredgecolor='none', markersize=5)
    plt.xlabel('Temperature')
    plt.ylabel('Energy')

    plt.subplot(2,2,2)
    plt.plot(temp, M, 'go', markeredgecolor='none', markersize=5)
    plt.xlabel('Temperature')
    plt.ylabel('Magnetization')

    plt.subplot(2,2,3)
    plt.plot(temp, C, 'mo', markeredgecolor='none', markersize=5)
    plt.xlabel('Temperature')
    plt.ylabel('Specific heat')

    plt.subplot(2,2,4)
    plt.plot(temp, X, 'bo', markeredgecolor='none', markersize=5)
    plt.xlabel('Temperature')
    plt.ylabel('Susceptibility')


    plt.tight_layout()
    ## uncomment if you want to save the data
    #np.savetxt("ising_2d_{}_{}_{}.dat".format(N,steps,H), np.array([temp, E, M , C,X]).transpose())

plt.show()

NameError: name 'snaps' is not defined

### TODO
gather all data and compare with $N$